In [1]:
"""
import zipfile
import os

# Path to the zip file
zip_path = 'TestDataSet.zip'

# Directory where you want to extract the contents
extract_to = 'TestDataSet'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f'Unzipped {zip_path} to {extract_to}/')
"""

"\nimport zipfile\nimport os\n\n# Path to the zip file\nzip_path = 'TestDataSet.zip'\n\n# Directory where you want to extract the contents\nextract_to = 'TestDataSet'\n\n# Unzip the file\nwith zipfile.ZipFile(zip_path, 'r') as zip_ref:\n    zip_ref.extractall(extract_to)\n\nprint(f'Unzipped {zip_path} to {extract_to}/')\n"

**Task 1: Basics**

In [2]:
import torch
import torchvision
from torchvision import transforms
from torchvision.models import resnet34
import json
import os
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
# Load the pretrained model
pretrained_model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
pretrained_model.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Mean and std for ImageNet normalization
mean_norms = np.array([0.485, 0.456, 0.406])
std_norms = np.array([0.229, 0.224, 0.225])

plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norms, std=std_norms)
])

In [5]:
# Load dataset
dataset_path = "/content/TestDataSet/TestDataSet"
dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=plain_transforms)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [6]:
# Load labels
with open(os.path.join(dataset_path, 'labels_list.json')) as f:
    labels_map = json.load(f)

# Build mapping from folder name to ImageNet index
# labels_list.json has format like "401: accordion"
imagenet_index_map = {line.split(":")[0]: int(line.split(":")[0]) for line in labels_map}

In [7]:
# Get folder names and build custom class index mapping
folder_to_imagenet_index = {}
class_to_idx = dataset.class_to_idx  # e.g., {'n02672831': 0, ..., 'n03042490': 99}
for folder_name, idx in class_to_idx.items():
    # The index is in the correct order; 401 + idx gives the true ImageNet index
    folder_to_imagenet_index[idx] = 401 + idx

In [8]:
top1_correct = 0
top5_correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(dataloader):
        images = images.cuda()
        labels = labels.cuda()

        outputs = pretrained_model(images)
        _, top5 = outputs.topk(5, dim=1)

        # Convert label indices to actual ImageNet indices
        actual_indices = torch.tensor([folder_to_imagenet_index[l.item()] for l in labels]).to(labels.device)

        # Top-1
        top1_preds = top5[:, 0]
        top1_correct += (top1_preds == actual_indices).sum().item()

        # Top-5
        for i in range(images.size(0)):
            if actual_indices[i].item() in top5[i].tolist():
                top5_correct += 1

        total += images.size(0)

top1_accuracy = top1_correct / total
top5_accuracy = top5_correct / total

print(f"\nTop-1 Accuracy: {top1_accuracy * 100:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy * 100:.2f}%")


100%|██████████| 16/16 [00:02<00:00,  6.54it/s]

Top-1 Accuracy: 76.00%
Top-5 Accuracy: 94.20%


**Task 2: Pixel-wise attacks**